In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 890kB 16.3MB/s 
     |████████████████████████████████| 3.0MB 20.1MB/s 
     |████████████████████████████████| 1.1MB 28.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=27d7e8e1bc7d2e1e58d4af5f278d686fa273d7bfa7398de57a58474d1919232d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, IterableDataset, DataLoader
import numpy
import os
import codecs
import random
import sys
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from transformers import *
from tqdm import tqdm
import itertools

In [ ]:
# # If there's a GPU available...
if torch.cuda.is_available():    

	# Tell PyTorch to use the GPU.    
	device = torch.device("cuda")

	print('There are %d GPU(s) available.' % torch.cuda.device_count())

	print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
	print('No GPU available, using the CPU instead.')
	device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
class TreeDataset(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]


def _label_node_index(node, n=0):
    node['index'] = n
    for child in node['c']:
        n += 1
        _label_node_index(child, n)


def _gather_node_attributes(node, key):
    features = [node[key]]
    for child in node['c']:
        features.extend(_gather_node_attributes(child, key))
    return features


def _gather_adjacency_list(node):
    adjacency_list = []
    for child in node['c']:
        adjacency_list.append([node['index'], child['index']])
        adjacency_list.extend(_gather_adjacency_list(child))

    return adjacency_list


def convert_tree_to_tensors(tree, tweet_id, device=device):
    # Label each node with its walk order to match nodes to feature tensor indexes
    # This modifies the original tree as a side effect
    _label_node_index(tree)

    features = _gather_node_attributes(tree, 'f')
    attention = _gather_node_attributes(tree, 'a')
    old_features = _gather_node_attributes(tree, 'k')
    labels = _gather_node_attributes(tree, 'l')
    tweetid = tweet_id
    root_label = [labels[0]]
    s_gt = [tree['class_gt']]
    adjacency_list = _gather_adjacency_list(tree)

    node_order, edge_order = calculate_evaluation_orders(adjacency_list, len(features))

    root_node = [0]

    return {
        'f': torch.tensor(features, dtype=torch.long),
        'a':torch.tensor(attention,  dtype=torch.float32),
        'k':torch.tensor(old_features, dtype=torch.float32),
        's_gt':torch.tensor(s_gt,dtype=torch.long),
        'l': torch.tensor(labels,  dtype=torch.float32),
        'root_l': torch.tensor(root_label, dtype=torch.float32),
        'root_n': torch.tensor(root_node,  dtype=torch.int64),
        'node_order': torch.tensor(node_order,  dtype=torch.int64),
        'adjacency_list': torch.tensor(adjacency_list,  dtype=torch.int64),
        'edge_order': torch.tensor(edge_order,  dtype=torch.int64),
        'tweet_id' : torch.tensor(tweet_id, dtype=torch.int64)
    }

def calculate_evaluation_orders(adjacency_list, tree_size):
    '''Calculates the node_order and edge_order from a tree adjacency_list and the tree_size.

    The TreeLSTM model requires node_order and edge_order to be passed into the model along
    with the node features and adjacency_list.  We pre-calculate these orders as a speed
    optimization.
    '''
    adjacency_list = numpy.array(adjacency_list)

    node_ids = numpy.arange(tree_size, dtype=int)

    node_order = numpy.zeros(tree_size, dtype=int)
    unevaluated_nodes = numpy.ones(tree_size, dtype=bool)
    # print(adjacency_list)
    if(len(adjacency_list)==0):
        return [0],[]
    parent_nodes = adjacency_list[:, 0]
    child_nodes = adjacency_list[:, 1]

    n = 0
    while unevaluated_nodes.any():
        # Find which child nodes have not been evaluated
        unevaluated_mask = unevaluated_nodes[child_nodes]

        # Find the parent nodes of unevaluated children
        unready_parents = parent_nodes[unevaluated_mask]

        # Mark nodes that have not yet been evaluated
        # and which are not in the list of parents with unevaluated child nodes
        nodes_to_evaluate = unevaluated_nodes & ~numpy.isin(node_ids, unready_parents)

        node_order[nodes_to_evaluate] = n
        unevaluated_nodes[nodes_to_evaluate] = False

        n += 1

    edge_order = node_order[parent_nodes]

    return node_order, edge_order


def batch_tree_input(batch):
    '''Combines a batch of tree dictionaries into a single batched dictionary for use by the TreeLSTM model.

    batch - list of dicts with keys ('f', 'node_order', 'edge_order', 'adjacency_list')
    returns a dict with keys ('f', 'node_order', 'edge_order', 'adjacency_list', 'tree_sizes')
    '''
    tree_sizes = [b['f'].shape[0] for b in batch]

    batched_features = torch.cat([b['f'] for b in batch])
    batched_attentions = torch.cat([b['a'] for b in batch])
    batched_old_features = torch.cat([b['k'] for b in batch])
    batched_node_order = torch.cat([b['node_order'] for b in batch])

    idx = 0
    root_li = []

    for b in batch:
        root_li.append(idx)
        idx += len(b['node_order'])

    batched_root = torch.tensor(root_li,dtype=torch.int64)

    batched_edge_order = torch.cat([b['edge_order'] for b in batch])

    batched_labels = torch.cat([b['l'] for b in batch])

    batched_root_labels = torch.cat([b['root_l'] for b in batch])
    batched_cont_labels = torch.cat([b['s_gt'] for b in batch])
    batched_adjacency_list = []
    offset = 0
    for n, b in zip(tree_sizes, batch):
        batched_adjacency_list.append(b['adjacency_list'] + offset)
        offset += n
    batched_adjacency_list = torch.cat(batched_adjacency_list)

    return {
        'f': batched_features,
        'a': batched_attentions,
        'k': batched_old_features,
        's_gt':batched_cont_labels,
        'node_order': batched_node_order,
        'edge_order': batched_edge_order,
        'adjacency_list': batched_adjacency_list,
        'tree_sizes': tree_sizes,
        'root_node': batched_root,
        'root_label': batched_root_labels,
        'l': batched_labels
    }


def unbatch_tree_tensor(tensor, tree_sizes):
    '''Convenience functo to unbatch a batched tree tensor into individual tensors given an array of tree_sizes.

    sum(tree_sizes) must equal the size of tensor's zeroth dimension.
    '''
    return torch.split(tensor, tree_sizes, dim=0)

In [ ]:
class TreeLSTM(torch.nn.Module):
	'''PyTorch TreeLSTM model that implements efficient batching.
	'''
	def __init__(self,model_name,trainable_layers, in_features, out_features,mode):
		'''TreeLSTM class initializer

		Takes in int sizes of in_features and out_features and sets up model Linear network layers.
		'''
		super().__init__()
		print("model intialising...")
		self.in_features = in_features
		self.out_features = out_features
		self.mode = mode

		self.model_name = model_name

		if model_name == 'BERT':
			self.BERT_model  = BertModel.from_pretrained("bert-base-cased")
		elif model_name == 'ROBERTA':
			self.BERT_model  = RobertaModel.from_pretrained("roberta-base")
		elif model_name == 'XLNET':
			self.BERT_model = XLNetModel.from_pretrained("xlnet-base-cased")
		elif model_name == 'T5':
			self.BERT_model = T5Model.from_pretrained("t5-base")
   
		for name,param in self.BERT_model.named_parameters():
			flag = False
			for num in trainable_layers:
				if 'layer.'+str(num)+'.' in name:
					param.requires_grad = True
					flag = True
					break
			if not flag:
				if 'pooler' in name or 'embedding' in name:
					param.requires_grad = True
				else:
					param.requires_grad = False

		self.W_iou = torch.nn.Linear(self.in_features, 3 * self.out_features)
		self.U_iou = torch.nn.Linear(self.out_features, 3 * self.out_features, bias=False)

		self.W_f = torch.nn.Linear(self.in_features, self.out_features)
		self.U_f = torch.nn.Linear(self.out_features, self.out_features, bias=False)
		self.fc = torch.nn.Linear(self.out_features,2)
  
		self.cont_fc1 = torch.nn.Linear(self.in_features,self.out_features)
		self.cont_fc2 = torch.nn.Linear(self.out_features,4) 
	
	def forward(self, features,attentions,old_features,node_order, adjacency_list, edge_order, root_node, root_label):
		'''Run TreeLSTM model on a tree data structure with node features

		Takes Tensors encoding node features, a tree node adjacency_list, and the order in which 
		the tree processing should proceed in node_order and edge_order.
		'''

		# Total number of nodes in every tree in the batch
		batch_size = node_order.shape[0]

		# Retrive device the model is currently loaded on to generate h, c, and h_sum result buffers
		device = next(self.parameters()).device

		# h and c states for every node in the batch
		# h - hidden state
		# c - memory state
		h = torch.zeros(batch_size, self.out_features, device=device)
		
		c = torch.zeros(batch_size, self.out_features, device=device)

		hidden_states,_ = self.BERT_model(input_ids=features,attention_mask=attentions)  

		if self.mode=="cls":
			output_vectors = hidden_states[:,0]
		if self.mode=="avg":
			input_mask_expanded = attentions.unsqueeze(-1).expand(hidden_states.size()).float()
			sum_embeddings = torch.sum(hidden_states * input_mask_expanded, 1)
			sum_mask = input_mask_expanded.sum(1)
			output_vectors= sum_embeddings / sum_mask
			
		output_vectors = torch.cat([output_vectors,old_features],axis=1)
  
		x = output_vectors[root_node,:]
		x = self.cont_fc1(x)
		x = self.cont_fc2(x)
		# x = torch.nn.functional.softmax(x,dim=1)

		for n in range(node_order.max() + 1):
			self._run_lstm(n, h, c, output_vectors, node_order, adjacency_list, edge_order)
  
		h_root = h[root_node, :]
		# h_root = torch.nn.functional.softmax(h_root, dim = 1)
		out = self.fc(h_root)
		# out = torch.nn.functional.softmax(out, dim = 1)
  
		return h, out, c, x

	def _run_lstm(self, iteration, h, c, features, node_order, adjacency_list, edge_order):
		'''Helper function to evaluate all tree nodes currently able to be evaluated.
		'''
		node_mask = node_order == iteration

		# edge_mask is a tensor of size E x 1
		edge_mask = edge_order == iteration

		x = features[node_mask, :]
		if iteration == 0:
			iou = self.W_iou(x)
		else:
			# adjacency_list is a tensor of size e x 2
			adjacency_list = adjacency_list[edge_mask, :]

			parent_indexes = adjacency_list[:, 0]
			child_indexes = adjacency_list[:, 1]

			# child_h and child_c are tensors of size e x 1
			child_h = h[child_indexes, :]
			child_c = c[child_indexes, :]

			# Add child hidden states to parent offset locations
			_, child_counts = torch.unique_consecutive(parent_indexes, return_counts=True)
			child_counts = tuple(child_counts)
			parent_children = torch.split(child_h, child_counts)
			parent_list = [item.sum(0) for item in parent_children]

			h_sum = torch.stack(parent_list)
			iou = self.W_iou(x) + self.U_iou(h_sum)


		# i, o and u are tensors of size n x M
		i, o, u = torch.split(iou, iou.size(1) // 3, dim=1)
		i = torch.sigmoid(i)
		o = torch.sigmoid(o)
		u = torch.tanh(u)

		if iteration == 0:
			c[node_mask, :] = i * u
		else:
			# f is a tensor of size e x M
			f = self.W_f(features[parent_indexes, :]) + self.U_f(child_h)
			f = torch.sigmoid(f)
			# fc is a tensor of size e x M
			fc = f * child_c

			# Add the calculated f values to the parent's memory cell state
			parent_children = torch.split(fc, child_counts)
			parent_list = [item.sum(0) for item in parent_children]

			c_sum = torch.stack(parent_list)
			c[node_mask, :] = i * u + c_sum

		h[node_mask, :] = o * torch.tanh(c[node_mask])


In [ ]:
def save_model(model, name, val_acc=0):
  state = {
      'model':model.state_dict(),
      'optimizer': optimizer.state_dict(),
      'val_acc': val_acc
  }
  torch.save(state, name)

def load_model(model, name):
  state = torch.load(name)
  model.load_state_dict(state['model'])
  optimizer.load_state_dict(state['optimizer'])
  print('Validation accuracy of the model is ', state.get('val_acc'))
  return state.get('val_acc')

In [ ]:
seed_val = 40
random.seed(seed_val)
numpy.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

path = "./drive/My Drive/"
IN_FEATURES = 808
OUT_FEATURES = 128
NUM_ITERATIONS = 10
BATCH_SIZE = 16
TRAINABLE_LAYERS = [0,1,2,3,4,5,6,7,8,9,10,11]
learning_rates = [2e-5,1e-5]
name = path + "mtl_ver+cont.pt"
name2 = path + "mtl_ver+cont_2.pt"
mode = 'cls'
MODEL_NAME = 'BERT'
if MODEL_NAME == 'BERT':
    tree_path = './drive/My Drive/PT_PHEME5_FeatBERT40_Depth5_maxR5_MTL/'
elif MODEL_NAME == 'ROBERTA':
    tree_path = './drive/My Drive/Parsed-Trees-Pad32_FeatROBERT40_Depth5_maxR5/'
elif MODEL_NAME == 'XLNET':
    tree_path = './drive/My Drive/Parsed-Trees-Pad32_FeatXLNET40_Depth5_maxR5/'
elif MODEL_NAME =='T5':
    tree_path = './drive/My Drive/Parsed-Trees-Pad32_FeatT540_Depth5_maxR5/'

In [ ]:
def testing(test_trees,model,epoch):
    print('Now Testing:', test_file)
    train_acc = 0
    total = 0
    predicted = []
    ground = []
    cont_pred = []
    cont_ground = []
    model.eval()
    prob = []
    pred =[]
    tweetid = []
    for test in test_trees:
        try:
            h_test,h_test_root,c,cont_out = model(
                    test['f'].to(device),
                    test['a'].to(device),
                    test['k'].to(device),
                    test['node_order'].to(device),
                    test['adjacency_list'].to(device),
                    test['edge_order'].to(device),
                    test['root_n'].to(device),
                    test['root_l'].to(device)
            )
        except Exception as e:
            print(e)
            continue
        true_label_vals = test['root_l'].to('cpu')
        pred_label_vals = h_test_root.detach().cpu()
        pred_v, pred_label = torch.max(pred_label_vals, 1)
        true_label = true_label_vals[0][1]
        predicted.append(pred_label)
        ground.append(true_label)
        
        cont_true_val = test['s_gt']
        cont_pred_vals = cont_out.detach().cpu()
        cont_v,cont_label = torch.max(cont_pred_vals, 1)
        cont_pred.append(cont_label)
        cont_ground.append(cont_true_val)
    print("=================Testing===================")
    print(test_file)
    print("Verification:")
    print(classification_report(ground,predicted,digits=5))
    print("Content Classification")
    print(classification_report(cont_ground,cont_pred,digits=5))

    return 0

In [ ]:
def train(model,tree_batch,mode="train"):
    err_count = 0
    loss = 0
    pred_label = []
    g_labels = []
    cont_pred_labels = []
    cont_gt = []


    h,h_root,c, cont_out = model(
        tree_batch['f'].to(device),
        tree_batch['a'].to(device),
        tree_batch['k'].to(device),
        tree_batch['node_order'].to(device),
        tree_batch['adjacency_list'].to(device),
        tree_batch['edge_order'].to(device),
        tree_batch['root_node'].to(device),
        tree_batch['root_label'].to(device)
    )
    root_labels = tree_batch['root_label'].to(device)
    pred_label_vals = h_root.detach().cpu()
    pred_v, pred_label = torch.max(pred_label_vals, 1)
    root = root_labels.to('cpu')
    g_labels = [t[1] for t in root]

    cont_labels = tree_batch['s_gt'].to(device)
    cont_label_vals = cont_out.detach().cpu()
    cont_v,cont_label = torch.max(cont_label_vals, 1)
    cont_gt = tree_batch['s_gt'] 

    loss = loss_function(h_root,cont_out,root_labels,cont_labels,0.5,0.5,False)

    optimizer.zero_grad()
    if mode=="train":
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    return loss,pred_label,g_labels,cont_label,cont_gt,err_count

In [ ]:
# files = ['charliehebdo.txt', 'ottawashooting.txt','germanwings-crash.txt','sydneysiege.txt'] 
# cont_tweets = {}
# for f in files:
#     f = codecs.open(path+"situational_tweets/"+f[:-4]+"_FOUR_CLEAN_ANNOTATE_110520.txt")
#     for line in f:
#         line = line.split("\t")
#         cont_tweets[int(line[1])] = int(line[8])-1

In [ ]:
def split_data(trees,frac):
    data = [[] for _ in range(4)]
    for tree in trees:
        data[int(tree['s_gt'].item())].append(tree)
    # length = [int(frac*len(data[i])) for i in range(4)]
    # val_li = []
    # val_li = [val_li+data[i][:length[i]] for i in range(4)]
    # random.shuffle(val_li)
    # train_li = []
    # train_li = [train_li+data[i][length[i]:] for i in range(4)]
    # random.shuffle(train_li)
    # return train_li[0],val_li[0]
    length = [int(frac*len(data[i])) for i in range(4)]
    print([len(data[i][:length[i]]) for i in range(4)])
    val_li = []
    for i in range(4):
        val_li.extend(data[i][:length[i]])
    random.shuffle(val_li)

    train_li = []
    for i in range(4):
        train_li.extend(data[i][length[i]:])
    random.shuffle(train_li)
    return train_li,val_li

In [ ]:
files = ['charliehebdo.txt', 'ottawashooting.txt','germanwings-crash.txt','sydneysiege.txt'] 

tree_li = {}
val_li = {}
y = []
for filename in files:
    err = 0
    count = 0
    input_file = codecs.open(tree_path + filename, 'r', 'utf-8')
    tree_li[filename]=[]
    for row in input_file:
        s = row.strip().split('\t')
        count += 1
        tweet_id = int(s[0])
        curr_tree = eval(s[1])
        try:
            # ,cont_tweets[tweet_id],
            curr_tensor = convert_tree_to_tensors(curr_tree,tweet_id)
        except Exception as e:
            err += 1
            print(e)
            continue
        y.append(int(curr_tree['class_gt']))
        tree_li[filename].append(curr_tensor)
    random.shuffle(tree_li[filename])
    tree_li[filename],val_li[filename] = split_data(tree_li[filename],0.1)
    input_file.close()
    print("errors ", err)
    print("file count ",count)
    print("filename",len(tree_li[filename]),len(val_li[filename]))

[20, 37, 17, 132]
errors  0
file count  2079
filename 1873 206
[18, 18, 17, 34]
errors  0
file count  890
filename 803 87
[7, 13, 11, 14]
errors  0
file count  469
filename 424 45
[15, 23, 22, 61]
errors  0
file count  1221
filename 1100 121


In [ ]:
# tree_li['charliehebdo.txt'][0]['s_gt'].item()

3.0

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
content_weight_dict = {}
for test_file in files:
    train_y = []
    for f in files:
        if f != test_file:
            for t in tree_li[f]:
                train_y.append(t['s_gt'][0].tolist())
    content_weight_dict[test_file] = torch.tensor(compute_class_weight('balanced',numpy.unique(train_y),train_y),device=device,dtype=torch.float32)

In [ ]:
verification_weight_dict = {}
for test_file in files:
    y = []
    for filename in files:
        if filename != test_file:
            for tree in tree_li[filename]:
                y.append(int(tree['root_l'].tolist()[0][1]))
    verification_weight_dict[test_file] = torch.tensor(compute_class_weight('balanced',numpy.unique(y),y)).to(device)

In [ ]:
content_weight_dict

{'charliehebdo.txt': tensor([1.5350, 1.1705, 1.2538, 0.5894], device='cuda:0'),
 'germanwings-crash.txt': tensor([1.8956, 1.3296, 1.8259, 0.4603], device='cuda:0'),
 'ottawashooting.txt': tensor([2.1392, 1.2675, 1.8422, 0.4544], device='cuda:0'),
 'sydneysiege.txt': tensor([1.8023, 1.2400, 1.8675, 0.4755], device='cuda:0')}

In [ ]:
verification_weight_dict

{'charliehebdo.txt': tensor([0.9576, 1.0463], device='cuda:0', dtype=torch.float64),
 'germanwings-crash.txt': tensor([0.7628, 1.4512], device='cuda:0', dtype=torch.float64),
 'ottawashooting.txt': tensor([0.7394, 1.5441], device='cuda:0', dtype=torch.float64),
 'sydneysiege.txt': tensor([0.7550, 1.4804], device='cuda:0', dtype=torch.float64)}

In [ ]:
def loss_function(logits1,logits2,glabel1,glabel2,lambda1,lambda2,flood):
	loss1 = loss_function1(logits1,glabel1)
	loss2 = loss_function2(logits2,glabel2)
	loss = lambda1*loss1 + lambda2*loss2
	if flood:
		loss = (loss-0.25).abs()+0.25
	return loss

In [ ]:
for learning_rate in learning_rates:
    print("learning rate = ",learning_rate)
    for test in files:
        model = TreeLSTM(MODEL_NAME,TRAINABLE_LAYERS, IN_FEATURES, OUT_FEATURES,mode=mode).train()
        model.cuda()
        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay = 0.01)
        # optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate,amsgrad=True)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',patience=2, verbose=True)
        loss_function1 = torch.nn.BCEWithLogitsLoss(weight = verification_weight_dict[test])
        loss_function2 = torch.nn.CrossEntropyLoss(weight = content_weight_dict[test])
        test_file = test
        model.train()
        print('Training Set:', set(files) - {test_file})
        test_trees = []
        train_trees = []
        val_data = []
        for filename in files:
            if filename == test:
                test_trees.extend(tree_li[filename])
                test_trees.extend(val_li[filename])
            else:
                curr_tree_dataset = TreeDataset(tree_li[filename])
                train_trees.extend(curr_tree_dataset)
                val_data.extend(TreeDataset(val_li[filename]))
        print(len(test_trees))
        print("size of training data",len(train_trees))
        print("training started....")
        prev_loss = 1
        prev_acc = 0
        for i in range(NUM_ITERATIONS):
            model.train()
            model.zero_grad() 
            total_loss = 0
            data_gen = []
            length = []

            train_gen = DataLoader(
                    train_trees,
                    collate_fn=batch_tree_input,
                    batch_size=BATCH_SIZE,
                    shuffle = True
                )
            val_gen = DataLoader(val_data,
                    collate_fn=batch_tree_input,
                    batch_size=BATCH_SIZE,
                    shuffle = True)
            j = 0
            avg_loss=0
            ground_labels = []
            predicted_labels = []
            cont_ground_labels = []
            cont_predicted_labels = []
            val_ground_labels = []
            val_predicted_labels= []
            val_cont_ground = []
            val_cont_predicted = []
            err_count = 0
            batch_no = 0
            for tree_batch in train_gen:
                loss,p_labels1,g_labels1,p_labels2,g_labels2,err = train(model,tree_batch,"train")
                err_count+=err
                if err!=1:
                    ground_labels.extend(g_labels1)
                    predicted_labels.extend(p_labels1)
                    cont_ground_labels.extend(g_labels2)
                    cont_predicted_labels.extend(p_labels2)
                    j = j+1
                    avg_loss += loss
                    total_loss += loss
                batch_no += 1
            print("validation started..",len(val_data))
            model.eval()
            val_avg_loss = 0
            val_j = 0
            with torch.no_grad():
                for batch in val_gen:
                    loss,p_labels1,g_labels1,p_labels2,g_labels2,err = train(model,batch,"eval")
                    err_count+=err
                    if err!=1:
                        val_ground_labels.extend(g_labels1)
                        val_predicted_labels.extend(p_labels1)
                        val_cont_ground.extend(g_labels2)
                        val_cont_predicted.extend(p_labels2)
                        val_j += 1
                        val_avg_loss += loss
            val_acc1 = accuracy_score(val_ground_labels,val_predicted_labels)
            val_acc2 = accuracy_score(val_cont_ground,val_cont_predicted)
            val_acc = val_acc1*0.5 + val_acc2*0.5
            scheduler.step(val_acc)
            if(prev_acc<=val_acc):
                save_model(model, name , val_acc)
                prev_acc = val_acc
            print("errors ",err_count)
            print('Iteration ', i)
            print('Training Loss: ',avg_loss/j)	
            print('Validation loss: ',val_avg_loss/val_j)

            print('verification training accuracy: ',accuracy_score(ground_labels,predicted_labels))
            print('verification validation accuracy: ',val_acc1)
            print('verification training confusion matrix: ',confusion_matrix(ground_labels, predicted_labels))
            print('Content classification training accuracy: ',accuracy_score(cont_ground_labels,cont_predicted_labels))
            print('Content classification validation accuracy: ',val_acc2)
            print('Content classification training confusion matrix: ',confusion_matrix(cont_ground_labels, cont_predicted_labels))

            ###############################################################################################################
            ##############################...................TESTING......................#################################
            ###############################################################################################################
            if (i+1)%5==0:
                with torch.no_grad():
                    save_model(model,name2,val_acc)
                    # test_model = TreeLSTM(TRAINABLE_LAYERS, IN_FEATURES, OUT_FEATURES,"cls").eval()
                    # test_model.cuda()
                    output = load_model(model,name)
                    ret_val = testing(test_trees,model,i+1)
                output = load_model(model,name2)

        print('Iteration ', i+1,' Loss: ',total_loss)
        print('Training and Testing Complete')

learning rate =  2e-05
model intialising...



Training Set: {'sydneysiege.txt', 'germanwings-crash.txt', 'ottawashooting.txt'}
2079
size of training data 2327
training started....
validation started.. 253
errors  0
Iteration  0
Training Loss:  tensor(0.8983, device='cuda:0', grad_fn=<DivBackward0>)
Validation loss:  tensor(0.7480, device='cuda:0')
verification training accuracy:  0.664804469273743
verification validation accuracy:  0.7786561264822134
verification training confusion matrix:  [[912 303]
 [477 635]]
Content classification training accuracy:  0.5384615384615384
Content classification validation accuracy:  0.6442687747035574
Content classification training confusion matrix:  [[181  89  86  23]
 [ 96 212 156  33]
 [ 61 116 220  67]
 [ 26  72 249 640]]
validation started.. 253
errors  0
Iteration  1
Training Loss:  tensor(0.7071, device='cuda:0', grad_fn=<DivBackward0>)
Validation loss:  tensor(0.6594, device='cuda:0')
verification training accuracy:  0.7228190803609797
verification validation accuracy:  0.7786561264822